In [1]:
using Pkg
Pkg.activate("/Users/ryansenne/Documents/Github/SSM/")
# Pkg.activate("/home/ryansenne/PycharmProjects/ssm_julia/")

  Activating project at `\Users\ryansenne\Documents\Github\SSM`


In [2]:
include("/Users/ryansenne/Documents/Github/SSM//src/SSM.jl")
# include("/home/ryansenne/PycharmProjects/ssm_julia/src/SSM.jl")
using Random
using .SSM
using Distributions

In [3]:
# generate data for a gaussian regression
Random.seed!(123)

n = 100
p = 2

X = randn(n, p-1)
X = [ones(n) X]

beta = [1.0, 2.0]
sigma = 1.0

Y = X*beta + rand(Normal(0, sigma), n)

# fit the model

regression = SSM.GaussianRegression()
SSM.fit!(regression, reshape(X[:, 2], n, 1), Y)

3.6564468790329445

In [4]:
regression

Main.SSM.GaussianRegression([0.9629113108791268, 2.1378927672195], 1.090533957478954, true)

In [8]:
SSM.least_squares(regression, reshape(X[:, 2], 100, 1), Y)

108.69301457802659

In [7]:
regression.β = [1, 2]

2-element Vector{Int64}:
 1
 2

In [5]:
using ForwardDiff

In [6]:
regression.β = [0., 0.]
regression.σ² = 1

1

In [8]:
-SSM.gradient!([0., 0.], regression, X, Y)

2-element Vector{Float64}:
 -171.93998874690695
 -358.0850749184984

In [9]:
function log_likelihood(β::Vector, X::AbstractArray, y::AbstractVector)
    # add intercept if specified
    X = hcat(ones(size(X, 1)), X)
    residuals = y - X * β
    return sum(residuals.^2)
end

log_likelihood (generic function with 1 method)

In [10]:
f(θ) = log_likelihood(θ, reshape(X[:, 2], n, 1), Y)

ForwardDiff.gradient(f, [0., 0.])

2-element Vector{Float64}:
 -171.93998874690695
 -358.08507491849844